In [1]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoModel, AutoTokenizer
from langchain.vectorstores.vearch import Vearch

# repalce to your local model path
model_path ="/data/zhx/zhx/langchain-ChatGLM_new/chatglm2-6b"  

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda(0)

/export/anaconda3/envs/vearch_cluster_langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 7/7 [00:06<00:00,  1.03it/s]


In [2]:
query = "你好!"
response, history = model.chat(tokenizer, query, history=[])
print(f"Human: {query}\nChatGLM:{response}\n")
query = "你知道凌波微步吗，你知道都有谁学会了吗?"
response, history = model.chat(tokenizer, query, history=history)
print(f"Human: {query}\nChatGLM:{response}\n")

Human: 你好!
ChatGLM:你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。

Human: 你知道凌波微步吗，你知道都有谁学会了吗?
ChatGLM:凌波微步是一种步伐，源自金庸先生的武侠小说《天龙八部》。段誉是学会凌波微步的人之一。



In [3]:
# Add your local knowledge files
file_path = "/data/zhx/zhx/langchain-ChatGLM_new/knowledge_base/天龙八部/lingboweibu.txt"#Your local file path"
loader = TextLoader(file_path,encoding="utf-8")
documents = loader.load()

# split text into sentences and embedding the sentences
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

#replace to your model path
embedding_path = '/data/zhx/zhx/langchain-ChatGLM_new/text2vec/text2vec-large-chinese'
embeddings = HuggingFaceEmbeddings(model_name=embedding_path)


No sentence-transformers model found with name /data/zhx/zhx/langchain-ChatGLM_new/text2vec/text2vec-large-chinese. Creating a new one with MEAN pooling.


In [4]:
#first add your document into vearch vectorstore
vearch_standalone = Vearch.from_documents(
    texts,embeddings,path_or_url="/data/zhx/zhx/langchain-ChatGLM_new/knowledge_base/valid_table",table_name="valid_table",flag=0)

print("***************after is cluster res*****************")

vearch_cluster = Vearch.from_documents(
    texts,embeddings,path_or_url="http://test-vearch-langchain-router.vectorbase.svc.ht1.n.jd.local",db_name="valid_table_new_2",table_name="valid_table_new_1",flag=1)


['3556d6e803ff4c42b0d30cd2bb452ff2', '0079e8720dd546a3a3392e73009c3697', '95534c7b67cc4a4eb8244ca8541ec2af']
***************after is cluster res*****************
['-8844872069836508090', '-6664697494158621762', '149322492689170423']


In [6]:
query = "你知道凌波微步吗，你知道都有谁会凌波微步?"
vearch_standalone_res=vearch_standalone.similarity_search(query, 3)
for idx,tmp in enumerate(vearch_standalone_res): 
    print(f"{'#'*20}第{idx+1}段相关文档{'#'*20}\n\n{tmp.page_content}\n")

# combine your local knowleadge and query 
context = "".join([tmp.page_content for tmp in vearch_standalone_res])
new_query = f"基于以下信息，尽可能准确的来回答用户的问题。背景信息:\n {context} \n 回答用户这个问题:{query}\n\n"
response, history = model.chat(tokenizer, new_query, history=[])
print(f"********ChatGLM:{response}\n")

print("***************************after is cluster res******************************")

query_c = "你知道凌波微步吗，你知道都有谁会凌波微步?"
cluster_res=vearch_cluster.similarity_search(query_c, 3)
for idx,tmp in enumerate(cluster_res): 
    print(f"{'#'*20}第{idx+1}段相关文档{'#'*20}\n\n{tmp.page_content}\n")

# combine your local knowleadge and query 
context_c = "".join([tmp.page_content for tmp in cluster_res])
new_query_c = f"基于以下信息，尽可能准确的来回答用户的问题。背景信息:\n {context_c} \n 回答用户这个问题:{query_c}\n\n"
response_c, history_c = model.chat(tokenizer, new_query_c, history=[])
print(f"********ChatGLM:{response_c}\n")

####################第1段相关文档####################

午饭过后，段誉又练“凌波微步”，走一步，吸一口气，走第二步时将气呼出，六十四卦走完，四肢全无麻痹之感，料想呼吸顺畅，便无害处。第二次再走时连走两步吸一口气，再走两步始行呼出。这“凌波微步”是以动功修习内功，脚步踏遍六十四卦一个周天，内息自然而然地也转了一个周天。因此他每走一遍，内力便有一分进益。

这般练了几天，“凌波微步”已走得颇为纯熟，不须再数呼吸，纵然疾行，气息也已无所窒滞。心意既畅，跨步时渐渐想到《洛神赋》中那些与“凌波微步”有关的句子：“仿佛兮若轻云之蔽月，飘飘兮若流风之回雪”，“竦轻躯以鹤立，若将飞而未翔”，“体迅飞凫，飘忽若神”，“动无常则，若危若安。进止难期，若往若还”。



百度简介

凌波微步是「逍遥派」独门轻功身法，精妙异常。

凌波微步乃是一门极上乘的轻功，所以列于卷轴之末，以易经八八六十四卦为基础，使用者按特定顺序踏着卦象方位行进，从第一步到最后一步正好行走一个大圈。此步法精妙异常，原是要待人练成「北冥神功」，吸人内力，自身内力已【颇为深厚】之后再练。

####################第2段相关文档####################

《天龙八部》第五回 微步縠纹生

卷轴中此外诸种经脉修习之法甚多，皆是取人内力的法门，段誉虽自语宽解，总觉习之有违本性，单是贪多务得，便非好事，当下暂不理会。

卷到卷轴末端，又见到了“凌波微步”那四字，登时便想起《洛神赋》中那些句子来：“凌波微步，罗袜生尘……转眄流精，光润玉颜。含辞未吐，气若幽兰。华容婀娜，令我忘餐。”曹子建那些千古名句，在脑海中缓缓流过：“秾纤得衷，修短合度，肩若削成，腰如约素。延颈秀项，皓质呈露。芳泽无加，铅华弗御。云髻峨峨，修眉连娟。丹唇外朗，皓齿内鲜。明眸善睐，靥辅承权。瑰姿艳逸，仪静体闲。柔情绰态，媚于语言……”这些句子用在木婉清身上，“这话倒也有理”；但如用之于神仙姊姊，只怕更为适合。想到神仙姊姊的姿容体态，“皎若太阳升朝霞，灼若芙蓉出绿波”，但觉依她吩咐行事，实为人生至乐，心想：“我先来练这‘凌波微步’，此乃逃命之妙法，非害人之手段也，练之有百利而无一害。”

####################第3段相关文档####################

《天

In [7]:
query = "你知道vearch是什么吗?"
response, history = model.chat(tokenizer, query, history=history)
print(f"Human: {query}\nChatGLM:{response}\n")

vearch_info = ["Vearch 是一款存储大语言模型数据的向量数据库，用于存储和快速搜索模型embedding后的向量，可用于基于个人知识库的大模型应用",
              "Vearch 支持OpenAI, Llama, ChatGLM等模型，以及LangChain库",
              "vearch 是基于C语言,go语言开发的，并提供python接口，可以直接通过pip安装"]
vearch_source=[{'source': '/data/zhx/zhx/langchain-ChatGLM_new/knowledge_base/tlbb/three_body.txt'},{'source': '/data/zhx/zhx/langchain-ChatGLM_new/knowledge_base/tlbb/three_body.txt'},{'source': '/data/zhx/zhx/langchain-ChatGLM_new/knowledge_base/tlbb/three_body.txt'}]
vearch_standalone.add_texts(vearch_info,vearch_source)

print("*****************after is cluster res********************")

vearch_cluster.add_texts(vearch_info,vearch_source)




Human: 你知道vearch是什么吗?
ChatGLM:是的，我知道 Vearch。Vearch 是一种基于矩阵分解的方法，用于解决线性方程组。它是一种高效的算法，特别适用于大规模线性方程组的求解。Vearch 算法将矩阵分解为基向量的叉积形式，这样可以快速地求解线性方程组。

['029aa75548ff4f85908d089888fa84f4', '9e13390d56ce4cf08880579b3f6b340f', 'f7ee0b61bfe0476e988c19559f36d16f']
*****************after is cluster res********************
['-6632005459476004618', '-79270890206033064', '7036574374942192674']


['-6632005459476004618', '-79270890206033064', '7036574374942192674']

In [8]:
query3 = "你知道vearch是什么吗?"
res1 = vearch_standalone.similarity_search(query3, 3)
for idx,tmp in enumerate(res1): 
    print(f"{'#'*20}第{idx+1}段相关文档{'#'*20}\n\n{tmp.page_content}\n")

context1 = "".join([tmp.page_content for tmp in res1])
new_query1 = f"基于以下信息，尽可能准确的来回答用户的问题。背景信息:\n {context1} \n 回答用户这个问题:{query3}\n\n"
response, history = model.chat(tokenizer, new_query1, history=[])
print(f"***************ChatGLM:{response}\n")

print("***************after is cluster res******************")

query3_c = "你知道vearch是什么吗?"
res1_c = vearch_standalone.similarity_search(query3_c, 3)
for idx,tmp in enumerate(res1_c): 
    print(f"{'#'*20}第{idx+1}段相关文档{'#'*20}\n\n{tmp.page_content}\n")

context1_C = "".join([tmp.page_content for tmp in res1_c])
new_query1_c = f"基于以下信息，尽可能准确的来回答用户的问题。背景信息:\n {context1_C} \n 回答用户这个问题:{query3_c}\n\n"
response_c, history_c = model.chat(tokenizer, new_query1_c, history=[])

print(f"***************ChatGLM:{response_c}\n")

####################第1段相关文档####################

Vearch 是一款存储大语言模型数据的向量数据库，用于存储和快速搜索模型embedding后的向量，可用于基于个人知识库的大模型应用

####################第2段相关文档####################

Vearch 支持OpenAI, Llama, ChatGLM等模型，以及LangChain库

####################第3段相关文档####################

vearch 是基于C语言,go语言开发的，并提供python接口，可以直接通过pip安装

***************ChatGLM:是的，我知道 Vearch。Varch 是一款基于 C 语言和 Go 语言开发的向量数据库，旨在存储和快速搜索模型嵌入后的向量，可用于基于个人知识库的大模型应用。Varch 支持 OpenAI、Llama 和 ChatGLM 等模型，并提供了 Python 接口，用户可以通过 pip 安装。Varch 的特点是高可扩展性、高性能和易于使用。

***************after is cluster res******************
####################第1段相关文档####################

Vearch 是一款存储大语言模型数据的向量数据库，用于存储和快速搜索模型embedding后的向量，可用于基于个人知识库的大模型应用

####################第2段相关文档####################

Vearch 支持OpenAI, Llama, ChatGLM等模型，以及LangChain库

####################第3段相关文档####################

vearch 是基于C语言,go语言开发的，并提供python接口，可以直接通过pip安装

***************ChatGLM:是的，Vearch是一款存储大语言模型数据的向量数据库，用于存储和快速搜索模型embedding后的向量。它支持OpenAI，Llama，ChatGLM等模型，并且可以与LangChain库集成。Varch

In [9]:
##delete and get function need to maintian  docids 
##your docid


res_d=vearch_standalone.delete(['029aa75548ff4f85908d089888fa84f4', '9e13390d56ce4cf08880579b3f6b340f', 'f7ee0b61bfe0476e988c19559f36d16f'])
print("delete vearch standalone docid",res_d)
query = "你知道vearch是什么吗?"
response, history = model.chat(tokenizer, query, history=[])
print(f"Human: {query}\nChatGLM:{response}\n")

res_cluster=vearch_cluster.delete(['-6632005459476004618', '-79270890206033064', '7036574374942192674'])
print("delete vearch cluster docid",res_cluster)
query_c = "你知道vearch是什么吗?"
response_c, history = model.chat(tokenizer, query_c, history=[])
print(f"Human: {query}\nChatGLM:{response_c}\n")





get_delet_doc=vearch_standalone.get(['029aa75548ff4f85908d089888fa84f4', '9e13390d56ce4cf08880579b3f6b340f', 'f7ee0b61bfe0476e988c19559f36d16f'])
print("after delete docid to query again:",get_delet_doc)
get_id_doc=vearch_standalone.get(['22f4c5e8af7541fb875b546872ef4e85', 'd796cdfe3ff7474e923bf87168f8788e', '738f7314b4134dd19e187b17968a379f'])
print("get existed docid",get_id_doc)

get_delet_doc=vearch_cluster.get(['-6632005459476004618', '-79270890206033064', '7036574374942192674'])
print("after delete docid to query again:",get_delet_doc)
get_id_doc=vearch_cluster.get(['6883964611486955620', '-3465968368475202129', '-4346376829616681721','3080506117063327914','7036574374942192674'])
print("get existed docid",get_id_doc)


delete vearch standalone docid True
Human: 你知道vearch是什么吗?
ChatGLM:Vearch是一种类似于"vex"的词汇,可能指的是一个数学或计算机科学中的符号或术语。在数学中,vearch可能指的是一个向量,其中每个元素都等于该向量在坐标轴上移动一个单位后的值。在计算机科学中,vearch可能指的是一个算法或数据结构,其中每个元素都包含有关另一个元素的信息。

 Vearch并不是一个常见的单词或术语,因此我无法提供更多信息。如果您有更多信息或上下文,可以提供更多信息,我将尽力回答您的问题。

delete vearch cluster docid True
Human: 你知道vearch是什么吗?
ChatGLM:Vearch是一种高斯分布的概率分布,也称为高斯连结分布(Gaussian linking distribution)。它是基于高斯分布的一个变体,用于建模多个变量之间的关联关系,如时间序列中的多个状态或者空间中的多个位置。

Vearch分布可以用于建模多个变量之间的因果关系,并且可以处理因变量是时间序列或实数的情况。它也被广泛用于金融和经济学等领域中,如股票价格和交易量等数据的建模。

after delete docid to query again: {}
get existed docid {}
after delete docid to query again: {}
get existed docid {}
